# Financial analysis

Financial analysis of model behavior in different time windows and different strategies.

### Packages

In [1]:
using LinearAlgebra;
using CSV;
using DataFrames;
using JuMP;
using SpecialFunctions;
using Ipopt;
using Plots;
using StatsBase;

### Data

In [3]:
#returns
tableReturns=CSV.File("dailyReturnsPeriods.csv");
DataReturns = DataFrame(tableReturns);

M=39 
L=3 

χ=[1/3 1/3 1/3]; #risk profile
α=[0.6 , 0.75,  0.9 ]; #confindence level

### Time windows

In [4]:
dataYear=groupby(DataReturns,:ano)#grouping by year
dataMonth=groupby(DataReturns,:periodo)#grouping by month 

#example of annual data
# year2007=dataYear[1]; 
# year2008=dataYear[2];
#  ...

#example of quarterly data
# quarterly_07_09=vcat(year2007,year2008,year2009);

###  Version 1 (no return constraint)

In [25]:
function MOD_V1(period)   
   N=size(period,1)
   returns=zeros(N,M)
   returns.=period[1:N,5:(M+4)]
   
    modelT1 = Model(Ipopt.Optimizer) 

    @variables(modelT1, begin
    1 ≥ x[1:M] ≥ 0
        u[1:N] ≥ 0
        y[1:L] 
        end)
    
    @constraint(modelT1,sum(x[i] for i=1:M)== 1);
    @constraint(modelT1, u[1]==0 );
    @constraint(modelT1,[t=2:N], u[t] ≥ u[t-1] - transpose(x)*returns[t,:]);
    #@constraint(modelT1, sum(transpose(x)*retornos[t,:] for t=1:N) ≥ δ );
   
    #function  f1--------------------------------
    η=2^(-13)        
    f1(a) = max(a,0)    
    df1(a)= 1/2*(a/(sqrt(a^2+η))+1)       
    d2f1(a)= η/(2*(η+a^2)^(3/2))    
    #----------------------------------------------
    
    JuMP.register(modelT1, :f1, 1, f1, df1,  d2f1)
    @NLobjective(modelT1, Min,  sum(χ[j]*(y[j]+ 1/((1-α[j])*N)*sum(f1(u[t]-y[j]) for t=1:N )) for j=1:L)); 

    optimize!(modelT1) 
   
    #results--------------------------------------------------
        weightT1= value.(x); 
        VaRT1= value.(y); 
        uT1=value.(u);
        mixedCVaRT1=round(objective_value(modelT1);digits=3);
        return1= sum(transpose(weightT1)*returns[t,:] for t=1:N); # cumulative return

         #max drawdown
         maxD=maximum(uT1);
         #average Drawdown 
         avgD=mean(uT1);
    
        return mixedCVaRT1 , return1
end

MOD_V1 (generic function with 1 method)

### Risk/return
Version 1 - Portfolio with quarterly return

In [10]:
#return2=zeros(45)

#for i in 1:10
#  return2[i]=MOD_V1(vcat(dataMonth[4*i-3],dataMonth[4*i-2],dataMonth[4*i-1],dataMonth[4*i]))[2];
#end 

#for i in 11:20
#  return2[i]=MOD_V1(vcat(dataMonth[4*i-3],dataMonth[4*i-2],dataMonth[4*i-1],dataMonth[4*i]))[2];
#end;

#for i in 21:30
#  return2[i]=MOD_V1(vcat(dataMonth[4*i-3],dataMonth[4*i-2],dataMonth[4*i-1],dataMonth[4*i]))[2];
#end

#for i in 31:40
#  return2[i]=MOD_V1(vcat(dataMonth[4*i-3],dataMonth[4*i-2],dataMonth[4*i-1],dataMonth[4*i]))[2];
#end

for i in 41:45
  return2[i]=MOD_V1(vcat(dataMonth[4*i-3],dataMonth[4*i-2],dataMonth[4*i-1],dataMonth[4*i]))[2];
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     3417
Number of nonzeros in Lagrangian Hessian.............:      343

Total number of variables............................:      127
                     variables with only lower bounds:       85
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:       84
        inequality constraints with only lower bounds:       84
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [11]:
return2

45-element Vector{Float64}:
  4.786993572118778e-5
  7.471506516036795e-6
  5.831878904304935e-6
  3.200679364048534e-6
 -0.011179346523438407
 -9.886905762734166e-8
  7.90071282976215e-6
  1.7616634854754076e-5
  2.1741062403910877e-5
  1.2999772404512013
  6.188478423563336e-6
  1.1302348966726861e-5
  0.29911559043704294
  ⋮
  0.15160581543611595
  1.110370851166445e-6
  0.17850520555517127
  0.15824453609955288
  0.08334816403197014
  0.29417529825909683
  0.01839938519525225
  0.44629376582994484
  0.3679722296434415
  0.3600169146864547
  0.09975228478262141
  0.06839093076419873

### Mixed CVaR

Version 1 - Portfolio with quarterly return

In [31]:
#mixedCVaR2=zeros(45)

#for i in 1:10
#  mixedCVaR2[i]=MOD_V1(vcat(dataMonth[4*i-3],dataMonth[4*i-2],dataMonth[4*i-1],dataMonth[4*i]))[1]
#end 

# for i in 11:20
#   mixedCVaR2[i]=MOD_V1(vcat(dataMonth[4*i-3],dataMonth[4*i-2],dataMonth[4*i-1],dataMonth[4*i]))[1]
# end 

# for i in 21:30
#   mixedCVaR2[i]=MOD_V1(vcat(dataMonth[4*i-3],dataMonth[4*i-2],dataMonth[4*i-1],dataMonth[4*i]))[1]
# end 

# for i in 31:40
#   mixedCVaR2[i]=MOD_V1(vcat(dataMonth[4*i-3],dataMonth[4*i-2],dataMonth[4*i-1],dataMonth[4*i]))[1]
# end

 for i in 41:45
  mixedCVaR2[i]=MOD_V1(vcat(dataMonth[4*i-3],dataMonth[4*i-2],dataMonth[4*i-1],dataMonth[4*i]))[1]
 end    

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     3417
Number of nonzeros in Lagrangian Hessian.............:      343

Total number of variables............................:      127
                     variables with only lower bounds:       85
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:       84
        inequality constraints with only lower bounds:       84
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [32]:
mixedCVaR2

45-element Vector{Float64}:
 0.008
 0.008
 0.008
 0.008
 0.019
 0.008
 0.008
 0.008
 0.008
 0.008
 0.008
 0.008
 0.008
 ⋮
 0.013
 0.008
 0.014
 0.016
 0.01
 0.012
 0.098
 0.017
 0.022
 0.019
 0.024
 0.022

In [33]:
results2=DataFrame(mixedCVaR= mixedCVaR2, returns=  return2)
CSV.write("RiskReturnPortfolio.csv",results2)

"C://Users//Rafaela//Documents//Tese_doutorado//Output//RiskReturnPortfolio.csv"

### Efficient frontier

#### Version 1 ( with return constraint)

In [12]:
function MOD_V1_R(period, δ)   
   N=size(period,1)
   returns=zeros(N,M)
   returns.=period[1:N,5:(M+4)]
   #----------------------------
 
    modelT1 = Model(Ipopt.Optimizer) 

    @variables(modelT1, begin
    1 ≥ x[1:M] ≥ 0
        u[1:N] ≥ 0
        y[1:L] 
        end)
    
    @constraint(modelT1,sum(x[i] for i=1:M)== 1);
    @constraint(modelT1, u[1]==0 );
    @constraint(modelT1,[t=2:N], u[t] ≥ u[t-1] - transpose(x)*returns[t,:]);
    @constraint(modelT1, sum(transpose(x)*returns[t,:] for t=1:N) ≥ δ );
   
    #function f1--------------------------------
    η=2^(-13)        #parameter
   
    f1(a) = max(a,0)    

    df1(a)= 1/2*(a/(sqrt(a^2+η))+1)       
    d2f1(a)= η/(2*(η+a^2)^(3/2))    
    #---------------------------------------------
    
    JuMP.register(modelT1, :f1, 1, f1, df1,  d2f1)
    @NLobjective(modelT1, Min,  sum(χ[j]*(y[j]+ 1/((1-α[j])*N)*sum(f1(u[t]-y[j]) for t=1:N )) for j=1:L)); 
    optimize!(modelT1) 
   
        #Results
        weightT1= value.(x); 
        VaRT1= value.(y); 
        uT1=value.(u);
        mixedCVaRT1=round(objective_value(modelT1);digits=5);
        return1= sum(transpose(weightT1)*returns[t,:] for t=1:N); #acumulative return
    
        return mixedCVaRT1 , return1
end

MOD_V1_R (generic function with 1 method)

### Efficient frontier in times of crisis
Second quarter of 2008

In [13]:
Quarter2_2008=vcat(dataMonth[17],dataMonth[18],dataMonth[19],dataMonth[20]) 
δ3=[0.04, 0.08, 0.12, 0.16, 0.2,0.22, 0.24]
return3=zeros(7)
risk3=zeros(7)

for i in 1:7
Aux=MOD_V1_R(Quarter2_2008, δ3[i]);
return3[i]=Aux[2];
risk3[i]=Aux[1];
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     3018
Number of nonzeros in Lagrangian Hessian.............:      339

Total number of variables............................:      126
                     variables with only lower bounds:       84
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:       84
        inequality constraints with only lower bounds:       84
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [14]:
risk3

7-element Vector{Float64}:
 0.02901
 0.04361
 0.05974
 0.07674
 0.0962
 0.11294
 0.14121

### Efficient frontier in no crisis period
Second quarter of 2013

In [15]:
Quarter2_2013=vcat(dataMonth[80],dataMonth[81],dataMonth[82],dataMonth[83]) 
δ4=[0.2, 0.32, 0.36, 0.4, 0.44, 0.48, 0.52]
return4=zeros(7)
risk4=zeros(7)

for i in 1:7
Aux4=MOD_V1_R(Quarter2_2013, δ4[i])
return4[i]=Aux4[2]
risk4[i]=Aux4[1]
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     3266
Number of nonzeros in Lagrangian Hessian.............:      343

Total number of variables............................:      127
                     variables with only lower bounds:       85
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:       85
        inequality constraints with only lower bounds:       85
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [16]:
risk4

7-element Vector{Float64}:
 0.01147
 0.01471
 0.01592
 0.01722
 0.01877
 0.02091
 0.02409

## Risk control : rebalancing

In [94]:
function MODMR(period)   
   N=size(period,1)
   returns=zeros(N,M)
   returns.=period[1:N,5:(M+4)]
   #----------------------------
   #δ=0.0

    modelT1 = Model(Ipopt.Optimizer) 

    @variables(modelT1, begin
    1 ≥ x[1:M] ≥ 0   
        u[1:N] ≥ 0
        y[1:L] 
        end)
    
    @constraint(modelT1,sum(x[i] for i=1:M)== 1);
    @constraint(modelT1, u[1]==0 );
    @constraint(modelT1,[t=2:N], u[t] ≥ u[t-1] - transpose(x)*returns[t,:]);
   # @constraint(modelT1, sum(transpose(x)*returns[t,:] for t=1:N) ≥ δ );
   
    #function f1--------------------------------
    η=2^(-13)        #parameter
   
    f1(a) = max(a,0)    
    df1(a)= 1/2*(a/(sqrt(a^2+η))+1)      
    d2f1(a)= η/(2*(η+a^2)^(3/2))   
    #-------------------------------------------------------
    JuMP.register(modelT1, :f1, 1, f1, df1,  d2f1)
    @NLobjective(modelT1, Min,  sum(χ[j]*(y[j]+ 1/((1-α[j])*N)*sum(f1(u[t]-y[j]) for t=1:N )) for j=1:L)); 

    optimize!(modelT1) 
   
        #results
        weightT1= value.(x); 
        VaRT1= value.(y); 
        uT1=value.(u);
        mixedCVaRT1=round(objective_value(modelT1);digits=3);
        returns1= sum(transpose(weightT1)*returns[t,:] for t=1:N);
        maxD=maximum(uT1);
    
        return mixedCVaRT1 , returns1 , weightT1, maxD
end

MODMR (generic function with 1 method)

Algorithm - Portfolio

In [77]:
LimitDrawdown=0.1;
drawdown5=zeros(45);
Risk5=zeros(45);
weight5 =ones(39)*1/39;

In [83]:
for i in 41:45
    returns51=vcat(dataMonth[4*i-3],dataMonth[4*i-2],dataMonth[4*i-1],dataMonth[4*i]);
    N=size(returns51,1)
    returns52=zeros(N,M)
    returns52.=returns51[1:N,5:(M+4)]
    
   if drawdown5[i-1] < LimitDrawdown
       cumulativeReturn=zeros(N);
       for j in 1:N
         cumulativeReturn[j]= sum(transpose(weight5)*returns52[t,:] for t=1:j)
       end
       d=zeros(N);
       for k in 1:N
         d[k]= maximum(cumulativeReturn[1:k])-cumulativeReturn[k]
       end
       drawdown5[i]=maximum(d)
   else
     Risk5[i], drawdown5[i], weight5= MODMR(returns51)[2], MODMR(returns51)[4], MODMR(returns51)[3]
    
   end
end

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       40
Number of nonzeros in inequality constraint Jacobian.:     3417
Number of nonzeros in Lagrangian Hessian.............:      343

Total number of variables............................:      127
                     variables with only lower bounds:       85
                variables with lower and upper bounds:       39
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:       84
        inequality constraints with only lower bounds:       84
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [84]:
drawdown5

45-element Vector{Float64}:
 0.0
 0.16265049720512825
 1.568489841493147e-6
 3.7638430842458567e-6
 0.033106215411231814
 1.2192776421351846e-5
 3.0597321464413233e-6
 2.6589242966056257e-6
 1.8666263008245859e-6
 0.1486236978317223
 1.5978627680188444e-6
 1.145925803236556e-6
 2.8382331457264236e-6
 ⋮
 0.024539112312233904
 0.10347550242430265
 0.020197510536663497
 0.06485338683298209
 0.04752657927019615
 0.08184437725589815
 0.40515959763974774
 0.022126992307259504
 0.06812671551969084
 0.11930171550540844
 0.04015792027721633
 0.1384270845542683

Algorithm - equal weights

In [89]:
drawdown6=zeros(45);
return6=zeros(45);
weight6 =ones(39)*1/39;

In [92]:
for i in 2:45
    returns61=vcat(dataMonth[4*i-3],dataMonth[4*i-2],dataMonth[4*i-1],dataMonth[4*i]);
    N=size(returns61,1)
    returns62=zeros(N,M)
    returns62.=returns61[1:N,5:(M+4)]
    
    cumulativeReturn=zeros(N);
    for j in 1:N
         cumulativeReturn[j]= sum(transpose(weight6)*returns62[t,:] for t=1:j)
    end
    d=zeros(N);
    for k in 1:N
         d[k]= maximum(cumulativeReturn[1:k])-cumulativeReturn[k]
    end
    return6[i]=cumulativeReturn[N] 
    drawdown6[i]=maximum(d)
end

In [93]:
drawdown6

45-element Vector{Float64}:
 0.0
 0.16265049720512825
 0.10406163099999997
 0.13540348287179488
 0.20713623284615382
 0.47709263928205137
 0.11123347438461541
 0.0648673757948718
 0.08608630889743589
 0.1017751992051282
 0.09215472138461539
 0.044825937205128216
 0.08148739130769231
 ⋮
 0.061507819205128225
 0.18414231625641028
 0.04813774466666666
 0.08055697817948715
 0.08209487202564103
 0.04548458087179486
 0.6337165321282051
 0.06933561156410259
 0.08593499782051281
 0.095864303974359
 0.14314250987179483
 0.1392018447948718

In [95]:
results3=DataFrame(DrawdownP= drawdown5, DrawdownEW=  drawdown6)
CSV.write("C://Users//Rafaela//Documents//Tese_doutorado//Output//Rebalancing.csv",results3)

"C://Users//Rafaela//Documents//Tese_doutorado//Output//Rebalancing.csv"